In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import skew, kurtosis
from tqdm import tqdm

In [4]:
df=pd.read_csv(r"C:\Users\Abhi9\Downloads\simple_straight.csv")

C:\Users\Abhi9\AppData\Local\Temp\ipykernel_8740\3455879184.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"C:\Users\Abhi9\Downloads\simple_straight.csv")


In [7]:
df2=pd.read_csv('top_100 accounts.csv')

In [ ]:
# x=df['Account'].value_counts().sort_values(ascending=False).iloc[].reset_index()['Account'].to_list()

In [ ]:
df2=df[df['Account'].isin(df2)]

In [26]:
df.head()

,Account,Timestamp IST,Size USD,Closed PnL,Coin
0,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:17:40,14322.80,0.0,BTC
1,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:18:37,53355.20,0.0,BTC
2,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:22:47,58228.41,0.0,BTC
3,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:27:21,7173.40,0.0,BTC
4,0x8250a7f9fca9c33e68eead8c05c1d58c8fd70d13,2023-11-09 04:27:28,36153.32,0.0,BTC


In [ ]:
df2 = df2.copy()
df2['Closed PnL'] = df2['Closed PnL'].replace('-', np.nan).astype(float)
df2['Timestamp IST'] = pd.to_datetime(y['Timestamp IST'])
df2 = df2.sort_values(['Account', 'Timestamp IST'])
# Calculate cumulative PnL
df2['Cumulative PnL'] = df2.groupby('Account')['Closed PnL'].cumsum()
df2['Month'] = df2['Timestamp IST'].dt.to_period('M')

In [22]:
def cluster_traders(df, n_clusters=4):
    """
    Cluster traders based on their performance characteristics
    
    Parameters:
    df (pd.DataFrame): Input dataframe containing trader data
    n_clusters (int): Number of clusters to create (default: 4)
    
    Returns:
    pd.DataFrame: DataFrame with cluster assignments and features
    """
    # Feature extraction function
    def extract_features(trader_data):
        # Get all monthly data points
        monthly_data = trader_data.groupby('Month').agg(
            Cumulative_PnL=('Cumulative PnL', 'last'),
            Timestamp=('Timestamp IST', 'last')
        ).reset_index()
        
        # Timeline features
        timeline_features = monthly_data.set_index('Timestamp')['Cumulative_PnL']
        
        # Trend features
        monthly_changes = timeline_features.diff().dropna()
        trend_stability = monthly_changes.std()
        
        # Drawdown features
        cumulative_max = timeline_features.cummax()
        drawdown = cumulative_max - timeline_features
        max_drawdown = drawdown.max()
        recovery_factor = timeline_features.iloc[-1] / max_drawdown if max_drawdown > 0 else 10
        
        # Trade distribution features
        trades = trader_data['Closed PnL'].dropna()
        n_trades = len(trades)
        
        if n_trades > 0:
            wins = trades > 0
            win_rate = wins.mean()
            win_sum = trades[wins].sum()
            loss_sum = trades[~wins].sum()
            profit_factor = win_sum / abs(loss_sum) if loss_sum < 0 else 10
            trade_skew = trades.skew() if n_trades > 2 else 0
            trade_kurt = trades.kurtosis() if n_trades > 3 else 0
        else:
            win_rate = profit_factor = trade_skew = trade_kurt = 0
        
        # Activity features
        trade_density = n_trades / len(monthly_data) if len(monthly_data) > 0 else 0
        
        return {
            'Cumulative_PnL': timeline_features.iloc[-1],  # Last cumulative value
            'Timeline_Length': len(monthly_data),  # Number of months in timeline
            'Trend_Stability': trend_stability,
            'Max_Drawdown': max_drawdown,
            'Recovery_Factor': recovery_factor,
            'Win_Rate': win_rate,
            'Profit_Factor': profit_factor,
            'Trade_Skewness': trade_skew,
            'Trade_Kurtosis': trade_kurt,
            'Trade_Density': trade_density
        }
    
    # Extract features for all accounts
    feature_data = []
    for account, trader_data in df.groupby('Account'):
        features = extract_features(trader_data)
        features['Account'] = account
        feature_data.append(features)
    
    feature_df = pd.DataFrame(feature_data)
    
    # Handle missing/infinite values - replace with median of the column
    numeric_cols = feature_df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        feature_df[col] = feature_df[col].replace([np.inf, -np.inf], np.nan)
        median_val = feature_df[col].median()
        feature_df[col] = feature_df[col].fillna(median_val)
    
    # Standardize features
    scaler = StandardScaler()
    features_to_scale = feature_df.drop('Account', axis=1)
    scaled_features = scaler.fit_transform(features_to_scale)
    
    # Perform clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    feature_df['Cluster'] = kmeans.fit_predict(scaled_features)
    
    # Add PCA components for visualization
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(scaled_features)
    feature_df['PC1'] = principal_components[:, 0]
    feature_df['PC2'] = principal_components[:, 1]
    
    return feature_df

def plot_clusters(feature_df):
    fig = px.scatter(
        feature_df, 
        x='PC1', 
        y='PC2', 
        color='Cluster',
        hover_name='Account',
        title='Trader Strategy Clusters',
        labels={'PC1': 'Strategy Pattern Component 1', 'PC2': 'Strategy Pattern Component 2'},
        hover_data=feature_df.columns
    )
    fig.update_traces(marker=dict(size=12, line=dict(width=2, color='DarkSlateGrey')))
    fig.show()

In [ ]:
clustered_data=cluster_traders(df2,n_clusters=7)

c:\Users\Abhi9\.jupyter\conda\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [24]:
clustered_data['Cluster'].value_counts().sort_values(ascending=False)

Cluster
5    43
2    21
0    18
4     9
3     5
6     2
1     1
Name: count, dtype: int64

In [57]:
df[df['Account'].isin(clustered_data[clustered_data['Cluster']==6]['Account'].to_list())]

,Unnamed: 0,Account,Timestamp IST,Size USD,Closed PnL,Coin,Cumulative PnL,Month
2774643,7690795,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:12:44,20113.64,0.0,HYPE,0.000000,2025-01
2774644,7690796,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:10,7090.03,0.0,HYPE,0.000000,2025-01
2774645,7690797,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:26,2646.35,0.0,HYPE,0.000000,2025-01
2774646,7690798,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:26,1186.20,0.0,HYPE,0.000000,2025-01
2774647,7690799,0x0087188d27ac73b715ad38dff9b99554861a2b55,2025-01-12 16:18:28,1769.96,0.0,HYPE,0.000000,2025-01
...,...,...,...,...,...,...,...,...
2246822,6368146,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,11.51,0.0,PENGU,146525.146499,2025-06
2246823,6368147,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,11.51,0.0,PENGU,146525.146499,2025-06
2246824,6368148,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,97.37,0.0,PENGU,146525.146499,2025-06
2246825,6368149,0xfef198f2d9782ca670e64a988fbbcfd3f1f15563,2025-06-30 16:51:14,122.26,0.0,PENGU,146525.146499,2025-06


In [25]:
merged_df=df.merge(right=clustered_data,on='Account')

In [62]:
merged_df.columns

Index(['Unnamed: 0', 'Account', 'Timestamp IST', 'Size USD', 'Closed PnL',
       'Coin', 'Cumulative PnL', 'Month', 'Cumulative_PnL', 'Timeline_Length',
       'Trend_Stability', 'Max_Drawdown', 'Recovery_Factor', 'Win_Rate',
       'Profit_Factor', 'Trade_Skewness', 'Trade_Kurtosis', 'Trade_Density',
       'Cluster', 'PC1', 'PC2'],
      dtype='object')

In [ ]:
# z=merged_df.groupby('Account')['Cluster'].value_counts().reset_index()

In [ ]:
# merged_df[merged_df['Account']=='0xb29fac73e9dfb6f02ea34abd553176c4d46e82bb']['Cluster']

275919     5
275920     5
275921     5
275922     5
275923     5
          ..
1960944    5
1960945    5
1960946    5
1960947    5
1960948    5
Name: Cluster, Length: 26633, dtype: int32

In [ ]:
# len(merged_df[merged_df['Cluster']==5]['Account'].unique())

43

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity


In [41]:
merged_df.head()

,Account,Timestamp IST,Size USD,Closed PnL,Coin,Cumulative_PnL,Timeline_Length,Trend_Stability,Max_Drawdown,Recovery_Factor,Win_Rate,Profit_Factor,Trade_Skewness,Trade_Kurtosis,Trade_Density,Cluster,PC1,PC2
0,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:40,7433.61,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.24181,1.671117,10.97511,1782.103925,3510.428571,5,0.259743,0.324938
1,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:44,41223.01,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.24181,1.671117,10.97511,1782.103925,3510.428571,5,0.259743,0.324938
2,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:45,41223.01,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.24181,1.671117,10.97511,1782.103925,3510.428571,5,0.259743,0.324938
3,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:46,41223.01,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.24181,1.671117,10.97511,1782.103925,3510.428571,5,0.259743,0.324938
4,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:47,1076.77,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.24181,1.671117,10.97511,1782.103925,3510.428571,5,0.259743,0.324938


In [42]:
numerical_features=['Size USD', 'Closed PnL', 'Cumulative PnL', 
    'Trend Stability', 'Max Drawdown', 'Recovery Factor', 'Win Rate',
    'Profit Factor', 'Trade Skewness', 'Trade Kurtosis', 'Trade Density'
    ]

In [52]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686418 entries, 0 to 2686417
Data columns (total 18 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Account          object 
 1   Timestamp IST    object 
 2   Size USD         float64
 3   Closed PnL       Float64
 4   Coin             object 
 5   Cumulative_PnL   float64
 6   Timeline_Length  int64  
 7   Trend_Stability  float64
 8   Max_Drawdown     float64
 9   Recovery_Factor  float64
 10  Win_Rate         float64
 11  Profit_Factor    float64
 12  Trade_Skewness   float64
 13  Trade_Kurtosis   float64
 14  Trade_Density    float64
 15  Cluster          int32  
 16  PC1              float64
 17  PC2              float64
dtypes: Float64(1), float64(12), int32(1), int64(1), object(3)
memory usage: 361.2+ MB


In [55]:
merged_df[['Timestamp IST', 'Size USD', 'Closed PnL',
       'Cumulative_PnL','Trend_Stability', 'Max_Drawdown',
       'Recovery_Factor', 'Win_Rate', 'Profit_Factor', 'Trade_Skewness',
       'Trade_Kurtosis', 'Trade_Density']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2686418 entries, 0 to 2686417
Data columns (total 12 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Timestamp IST    datetime64[ns]
 1   Size USD         float64       
 2   Closed PnL       Float64       
 3   Cumulative_PnL   float64       
 4   Trend_Stability  float64       
 5   Max_Drawdown     float64       
 6   Recovery_Factor  float64       
 7   Win_Rate         float64       
 8   Profit_Factor    float64       
 9   Trade_Skewness   float64       
 10  Trade_Kurtosis   float64       
 11  Trade_Density    float64       
dtypes: Float64(1), datetime64[ns](1), float64(10)
memory usage: 248.5 MB


In [50]:
merged_df['Closed PnL']=merged_df['Closed PnL'].replace('-',np.nan).astype('Float64')

In [54]:
merged_df['Timestamp IST']=pd.to_datetime(merged_df['Timestamp IST'])

In [57]:
account_features = merged_df.groupby('Account')[['Size USD', 'Closed PnL',
       'Cumulative_PnL','Trend_Stability', 'Max_Drawdown',
       'Recovery_Factor', 'Win_Rate', 'Profit_Factor', 'Trade_Skewness',
       'Trade_Kurtosis', 'Trade_Density']].mean()

# Standardize the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(account_features)

# Calculate cosine similarity
cosine_sim = cosine_similarity(scaled_features)

In [58]:
similarity_df = pd.DataFrame(
    cosine_sim,
    index=account_features.index,
    columns=account_features.index
)

# Get top 10 similar accounts to the target account
target_account = '0xb29fac73e9dfb6f02ea34abd553176c4d46e82bb'
top_similar_accounts = similarity_df[target_account].sort_values(ascending=False)[1:11]  # [1:] to exclude self

In [68]:
t=list(pd.DataFrame(top_similar_accounts.reset_index())['Account'].unique())

In [71]:
clustered_data[clustered_data['Account'].isin(t)]['Cluster'].value_counts()

Cluster
5    5
4    2
3    2
2    1
Name: count, dtype: int64

In [72]:
merged_df[merged_df['Account'].isin(t)]

,Account,Timestamp IST,Size USD,Closed PnL,Coin,Cumulative_PnL,Timeline_Length,Trend_Stability,Max_Drawdown,Recovery_Factor,Win_Rate,Profit_Factor,Trade_Skewness,Trade_Kurtosis,Trade_Density,Cluster,PC1,PC2
0,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:40,7433.61,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.241810,1.671117,10.975110,1782.103925,3510.428571,5,0.259743,0.324938
1,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:44,41223.01,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.241810,1.671117,10.975110,1782.103925,3510.428571,5,0.259743,0.324938
2,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:45,41223.01,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.241810,1.671117,10.975110,1782.103925,3510.428571,5,0.259743,0.324938
3,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:46,41223.01,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.241810,1.671117,10.975110,1782.103925,3510.428571,5,0.259743,0.324938
4,0x613b66e9c81c83698c2892af33e2d9566393a66d,2024-12-07 19:39:47,1076.77,0.0,DOGE,1.054650e+06,7,326535.793291,85923.278788,12.274319,0.241810,1.671117,10.975110,1782.103925,3510.428571,5,0.259743,0.324938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2438278,0xa312114b5795dff9b8db50474dd57701aa78ad1e,2025-06-30 16:24:36,1999.96,1487.64208,DOGE,2.736392e+06,2,50538.750325,0.000000,10.000000,0.497041,13.585929,-7.444756,492.982161,13688.000000,4,-2.698049,0.281302
2438279,0xa312114b5795dff9b8db50474dd57701aa78ad1e,2025-06-30 16:48:36,257.80,190.42639,DOGE,2.736392e+06,2,50538.750325,0.000000,10.000000,0.497041,13.585929,-7.444756,492.982161,13688.000000,4,-2.698049,0.281302
2438280,0xa312114b5795dff9b8db50474dd57701aa78ad1e,2025-06-30 16:48:36,1116.74,824.89638,DOGE,2.736392e+06,2,50538.750325,0.000000,10.000000,0.497041,13.585929,-7.444756,492.982161,13688.000000,4,-2.698049,0.281302
2438281,0xa312114b5795dff9b8db50474dd57701aa78ad1e,2025-06-30 16:49:40,2000.03,1479.24336,DOGE,2.736392e+06,2,50538.750325,0.000000,10.000000,0.497041,13.585929,-7.444756,492.982161,13688.000000,4,-2.698049,0.281302


In [59]:
print("Top 10 most similar accounts to", target_account)
print(top_similar_accounts)

Top 10 most similar accounts to 0xb29fac73e9dfb6f02ea34abd553176c4d46e82bb
Account
0x47a761bb9e970ac93cb571c4614c4ca643714e4f    0.889356
0x863b676e5e4fea0541062c32983dc8f84749ca6d    0.841046
0x613b66e9c81c83698c2892af33e2d9566393a66d    0.709713
0xb83de012dba672c76a7dbbbf3e459cb59d7d6e36    0.683623
0xe867fbdad3291530e41530301ecb77693850c78e    0.603577
0x8e096995c3e4a3f0bc5b3ea1cba94de2aa4d70c9    0.598525
0xcb92c5988b1d4f145a7b481690051f03ead23a13    0.586086
0x565a65432ca44a999eb7217815d58594a559ccb2    0.568159
0x081942c75ff82341ab9f71d5df8b1309f6ddbc3e    0.556027
0xa312114b5795dff9b8db50474dd57701aa78ad1e    0.535077
Name: 0xb29fac73e9dfb6f02ea34abd553176c4d46e82bb, dtype: float64


In [ ]:
merged_df[merged_df['Account']!='0xb29fac73e9dfb6f02ea34abd553176c4d46e82bb'].groupby('Account').mean()

In [65]:
merged_df.columns

Index(['Account', 'Timestamp IST', 'Size USD', 'Closed PnL', 'Coin',
       'Cumulative PnL', 'Month', 'Cumulative_PnL', 'Timeline_Length',
       'Trend_Stability', 'Max_Drawdown', 'Recovery_Factor', 'Win_Rate',
       'Profit_Factor', 'Trade_Skewness', 'Trade_Kurtosis', 'Trade_Density',
       'Cluster', 'PC1', 'PC2'],
      dtype='object')

In [70]:
for i in  range(0,7):
    print(i)
    merged_df[merged_df['Cluster']==i][['Account', 'Timestamp IST', 'Size USD', 'Closed PnL', 'Coin',
        'Cumulative PnL',
        'Trend_Stability', 'Max_Drawdown', 'Recovery_Factor', 'Win_Rate',
        'Profit_Factor', 'Trade_Skewness', 'Trade_Kurtosis', 'Trade_Density']].to_csv(f"Cluster {i}.csv")

0
1
2
3
4
5
6


In [73]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5875990 entries, 0 to 5875989
Data columns (total 20 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Account          object        
 1   Timestamp IST    datetime64[ns]
 2   Size USD         float64       
 3   Closed PnL       float64       
 4   Coin             object        
 5   Cumulative PnL   float64       
 6   Month            period[M]     
 7   Cumulative_PnL   float64       
 8   Timeline_Length  int64         
 9   Trend_Stability  float64       
 10  Max_Drawdown     float64       
 11  Recovery_Factor  float64       
 12  Win_Rate         float64       
 13  Profit_Factor    float64       
 14  Trade_Skewness   float64       
 15  Trade_Kurtosis   float64       
 16  Trade_Density    float64       
 17  Cluster          int32         
 18  PC1              float64       
 19  PC2              float64       
dtypes: datetime64[ns](1), float64(14), int32(1), int64(1), object(2), pe